## Import the COCO dataset

In [5]:
!git clone https://github.com/cocodataset/cocoapi.git

Cloning into 'cocoapi'...


In [2]:
cd cocoapi/PythonAPI

C:\data\Rachel\Junior Year\Summer\cocoapi\PythonAPI


In [3]:
!python setup.py build

running build
running build_py
creating build
creating build\lib.win-amd64-3.7
creating build\lib.win-amd64-3.7\pycocotools
copying pycocotools\coco.py -> build\lib.win-amd64-3.7\pycocotools
copying pycocotools\cocoeval.py -> build\lib.win-amd64-3.7\pycocotools
copying pycocotools\mask.py -> build\lib.win-amd64-3.7\pycocotools
copying pycocotools\__init__.py -> build\lib.win-amd64-3.7\pycocotools
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools\_mask.c
building 'pycocotools._mask' extension


C:\Users\picni\Anaconda3\lib\site-packages\Cython\Compiler\Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: C:\data\Rachel\Junior Year\Summer\cocoapi\PythonAPI\pycocotools\_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
error: Microsoft Visual C++ 14.0 is required. Get it with "Microsoft Visual C++ Build Tools": https://visualstudio.microsoft.com/downloads/


In [4]:
!python setup.py install

running install
running bdist_egg
running egg_info
creating pycocotools.egg-info
writing pycocotools.egg-info\PKG-INFO
writing dependency_links to pycocotools.egg-info\dependency_links.txt
writing requirements to pycocotools.egg-info\requires.txt
writing top-level names to pycocotools.egg-info\top_level.txt
writing manifest file 'pycocotools.egg-info\SOURCES.txt'
reading manifest file 'pycocotools.egg-info\SOURCES.txt'
writing manifest file 'pycocotools.egg-info\SOURCES.txt'
installing library code to build\bdist.win-amd64\egg
running install_lib
running build_py
running build_ext
skipping 'pycocotools\_mask.c' Cython extension (up-to-date)
building 'pycocotools._mask' extension


error: Microsoft Visual C++ 14.0 is required. Get it with "Microsoft Visual C++ Build Tools": https://visualstudio.microsoft.com/downloads/
